This is the first look at the both sides of the conversation - replies and their originals from relevant users (found through the keyword search)

In [143]:
import pandas as pd
import numpy as np
import matplotlib
from collections import Counter
from scipy.stats import lognorm

%matplotlib inline
import matplotlib.pyplot as plt

This is the data for the users who: 
* replied to at least one tweet from a relevant user (user found through keyword search)
* were replied to and were also a relevant user (the other side)

In [107]:
df=pd.read_csv('../data/bbContextOntopicRepliesBothSidesDF.csv',sep=';')

In [43]:
len(df)

2535246

In [44]:
len(df.user_id.unique())

10011

Number of users is consistent with Contextual_GetUsersBothSidesReplies where this list was collated

In [45]:
df.columns

Index([u'id', u'username', u'user_id', u'geo_enabled', u'followCount',
       u'friendCount', u'faveCount', u'tweetCount', u'date', u'timing',
       u'coordinates', u'location', u'source', u'retweetCount', u'isRetweet',
       u'origTweet', u'origUser', u'origTime', u'origPlace',
       u'origUserCreated', u'contributors', u'hashtags', u'has_media', u'urls',
       u'mentions', u'replyWho', u'replyWhat', u'text'],
      dtype='object')

In [325]:
repliedToTweets=(df.replyWhat[df.replyWhat!=0]).unique()

In [326]:
len(repliedToTweets)

599658

In [327]:
float(Out[7])/Out[3]

0.236528920664898

So almost a quarter of the local contextual tweets is Replies

*Compare with proportion of replies in overall bb data set (later)*

In [130]:
len((df.user_id[df.replyWho!='0']).unique())

9375

* In Contextual_GetUsersBothSidesReplies, there were only 7207 repliers who responded to a tweet from relevant user also in bb
* This means that another 2168 users with replies in this data set are part of 2804 users who were replied to
* So a good proportion of them (77%) themselevs had replies

In [158]:
bbRepliersOnly=np.fromfile('/home/mako0970/ReplyPaperNew/data/bbContextRepliers.csv', sep=',', dtype=int)

In [160]:
bbRepliers=list(set(bbRepliersOnly) & set(df.user_id.unique()))

In [164]:
len(bbRepliers)

7207

In [328]:
repToTweetsInDF=list(set(repliedToTweets)& set(df.id))

In [329]:
len(repToTweetsInDF)

9885

Only about 10K tweets that local users replied to are by other local users

Maybe instead of looking for the original tweets by locals I should broaden to the entire East Coast?

In [166]:
bbRepOrigTweets=(df.replyWhat[(df.replyWho!=0) & (df.user_id.isin(bbRepliers))]).unique()

In [167]:
len(list(set(bbRepOrigTweets)& set(df.id)))

7467

So the 7207 bb ontopic repliers produced 7467 replies to tweets that were also in this data set (by ontopic bb users)

In [168]:
bbReplBS=np.fromfile('/home/mako0970/ReplyPaperNew/data/bbContextRepliesBothSidesUserIDs.csv', sep=',', dtype=int)

In [171]:
bbOrigUsers=list(set(bbReplBS) - set(bbRepliers))

In [172]:
len(bbOrigUsers)

2804

2804 of the replies were generated by the original authors - users whose tweets were replied to in the keyword set

In [173]:
bbOrigUsersOrigTweets=(df.replyWhat[(df.replyWho!=0) & (df.user_id.isin(bbOrigUsers))]).unique()

In [174]:
len(list(set(bbOrigUsersOrigTweets)& set(df.id)))

2453

In [175]:
len((set(bbRepOrigTweets)& set(df.id)) | (set(bbOrigUsersOrigTweets)& set(df.id)))

9885

So all the replies were produced by either repliers or the users whose tweets were repled to (who didn't have to have any replies by design -> <10011 tweets)

# Add info about the original tweets to DF

In [111]:
df['replyOrigTime']=0
df['replyOrigText']=0
df['replyOrigReplyWhat']=0
df['replyOrigReplyWho']=0

In [112]:
for id in repToTweetsInDF:
    df.loc[df.replyWhat==id, 'replyOrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.replyWhat==id, 'replyOrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.replyWhat==id, 'replyOrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.replyWhat==id, 'replyOrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [177]:
len((df[df.replyOrigTime!=0])

10373

10373 tweets replied to 9885 distinct tweets. So some originals had more than one reply

In [179]:
len(df[df.replyOrigReplyWhat!=0])

5272

So more than half of tweets that were replied to are themselves replies

In [181]:
len(df[(df.replyOrigReplyWho!=0)& (df.replyOrigReplyWho!='0')])

5973

701 originals that are themselves replies have usernames but not tweets IDs? **Why?**

In [186]:
df[(df.replyOrigReplyWho!=0)& (df.replyOrigReplyWho!='0') & (df.replyOrigReplyWhat==0)]

,id,username,user_id,geo_enabled,followCount,friendCount,faveCount,tweetCount,date,timing,...,has_media,urls,mentions,replyWho,replyWhat,text,replyOrigTime,replyOrigText,replyOrigReplyWhat,replyOrigReplyWho
4802,256938585092157440,DanKloefflerABC,22239799,True,1174,634,8,1139,2012-10-13 02:05:07,before,...,False,[],[u'LindseySnell'],LindseySnell,256931226965905408,@LindseySnell Thank you! But can I trade that ...,2012-10-13 01:35:52,@DanKloefflerABC I'm so proud of you!,0,DanKloefflerABC
8799,256971531786285057,DDougyy,26957356,False,546,494,100,20315,2012-10-13 04:16:02,before,...,False,[],"[u'Megglezzz', u'AlbanyDevils']",Megglezzz,256969224558362624,@Megglezzz @albanydevils I want this info too!,2012-10-13 04:06:52,@AlbanyDevils where can one watch/listen to to...,0,AlbanyDevils
13430,257086128446906368,terrymarley,24914924,True,584,676,36,5289,2012-10-13 11:51:24,before,...,False,[u'http://philadelphia.cbslocal.com/2012/10/12...,[u'DalilaRobin'],DalilaRobin,257083385405661184,@DalilaRobin Tell them this is CAPA Alumni wee...,2012-10-13 11:40:30,"@terrymarley, I am beating the drum. You want ...",0,terrymarley
13457,257086614071816192,terrymarley,24914924,True,584,676,36,5289,2012-10-13 11:53:19,before,...,False,[],[u'DalilaRobin'],DalilaRobin,257083385405661184,@DalilaRobin Please have a peaceful weekend an...,2012-10-13 11:40:30,"@terrymarley, I am beating the drum. You want ...",0,terrymarley
23479,257180555429437440,PatPizzy,479270769,True,128,121,332,5942,2012-10-13 18:06:37,before,...,False,[],[u'ADaza91'],ADaza91,257178830119243777,@ADaza91 Ighttttttt,2012-10-13 17:59:45,@PatPizzy yo when u get a chance watch this h...,0,PatPizzy
28142,257220844181389312,FrankRamblings,16857622,True,9686,518,1816,88275,2012-10-13 20:46:42,before,...,False,[],[u'MallorieRose'],MallorieRose,257217707152379905,@MallorieRose wow! Ill have to get it from you...,2012-10-13 20:34:14,@frankramblings homework already!,0,FrankRamblings
36937,257292202869391360,jhdemicco,9916742,True,429,417,8,486,2012-10-14 01:30:16,before,...,False,[],"[u'bjdu4', u'teresalabarbera']",bjdu4,257269414070722561,@bjdu4 @teresalabarbera You guys look great. C...,2012-10-13 23:59:42,@teresalabarbera purpling out at a dinner in h...,0,teresalabarbera
37376,257295437688614912,maryskribz,398707167,True,280,253,4574,6498,2012-10-14 01:43:07,before,...,False,[],[],tconleey,257295177146843137,@tomconl3y got in trouble so I went home :(,2012-10-14 01:42:05,@maryskribz Where are yew,0,maryskribz
39544,257310275651383296,SGottiR,187969481,True,131,249,519,8408,2012-10-14 02:42:04,before,...,False,[],"[u'SGottiR', u'alexrodriguezNY', u'NickSwisher...",SGottiR,257309882582192128,@SGottiR @alexrodriguezNY @NickSwisher @cgrand...,2012-10-14 02:40:31,@alexrodriguezNY @NickSwisher @cgrand14 @Robin...,0,alexrodriguezNY
39558,257310426839257090,SGottiR,187969481,True,131,249,519,8408,2012-10-14 02:42:41,before,...,False,[],"[u'SGottiR', u'alexrodriguezNY', u'NickSwisher...",SGottiR,257309882582192128,@SGottiR @alexrodriguezNY @NickSwisher @cgrand...,2012-10-14 02:40:31,@alexrodriguezNY @NickSwisher @cgrand14 @Robin...,0,alexrodriguezNY


In [190]:
origOrigTweets=list(set(df.replyOrigReplyWhat[df.replyOrigReplyWhat!=0]) & set(df.id))

In [191]:
len(origOrigTweets)

3673

3673 of these originals that are themselves replies are responding to the tweets that are also in the data set

# Add info about the second level of replied to tweets to the DF

In [194]:
df['reply2OrigTime']=0
df['reply2OrigText']=0
df['reply2OrigReplyWhat']=0
df['reply2OrigReplyWho']=0

In [196]:
for id in origOrigTweets:
    df.loc[df.replyOrigReplyWhat==id, 'reply2OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.replyOrigReplyWhat==id, 'reply2OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.replyOrigReplyWhat==id, 'reply2OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.replyOrigReplyWhat==id, 'reply2OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [197]:
len(df[df.reply2OrigTime!=0])

3925

In [200]:
len(df[(df.reply2OrigReplyWho!=0) & (df.reply2OrigReplyWho!='0')])

2526

In [201]:
len(df[df.reply2OrigReplyWhat!=0])

2301

In [210]:
len(df[(df.reply2OrigReplyWho!=0) & (df.reply2OrigReplyWho!='0') & (df.reply2OrigReplyWhat==0)])

225

So again, we have 2526 original originals that are themselves replies, but 225 of those only have the user, not the tweet id of their originals

In [213]:
orig3Tweets=list(set(df.reply2OrigReplyWhat[df.reply2OrigReplyWhat!=0]) & set(df.id))

In [214]:
len(orig3Tweets)

1893

And 1893 of those are in the data set

# Add info about the 3rd level of replied to tweets to the DF

In [212]:
df['reply3OrigTime']=0
df['reply3OrigText']=0
df['reply3OrigReplyWhat']=0
df['reply3OrigReplyWho']=0

In [215]:
for id in orig3Tweets:
    df.loc[df.reply2OrigReplyWhat==id, 'reply3OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply2OrigReplyWhat==id, 'reply3OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply2OrigReplyWhat==id, 'reply3OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply2OrigReplyWhat==id, 'reply3OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [216]:
len(df[df.reply3OrigTime!=0])

2040

In [217]:
len(df[(df.reply3OrigReplyWho!=0) & (df.reply3OrigReplyWho!='0')])

1403

In [218]:
len(df[df.reply3OrigReplyWhat!=0])

1315

In [221]:
len(df[(df.reply3OrigReplyWho!=0) & (df.reply3OrigReplyWho!='0') & (df.reply3OrigReplyWhat==0)])

88

In [231]:
orig4Tweets=list(set(df.reply3OrigReplyWhat[df.reply3OrigReplyWhat!=0]) & set(df.id))

In [232]:
len(orig4Tweets)

1121

# Add info about the 4th level of replied to tweets to the DF

In [233]:
df['reply4OrigTime']=0
df['reply4OrigText']=0
df['reply4OrigReplyWhat']=0
df['reply4OrigReplyWho']=0

In [234]:
for id in orig4Tweets:
    df.loc[df.reply3OrigReplyWhat==id, 'reply4OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply3OrigReplyWhat==id, 'reply4OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply3OrigReplyWhat==id, 'reply4OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply3OrigReplyWhat==id, 'reply4OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [235]:
len(df[df.reply4OrigTime!=0])

1216

In [236]:
len(df[(df.reply4OrigReplyWho!=0) & (df.reply4OrigReplyWho!='0')])

897

In [237]:
len(df[df.reply4OrigReplyWhat!=0])

841

In [239]:
orig5Tweets=list(set(df.reply4OrigReplyWhat[df.reply4OrigReplyWhat!=0]) & set(df.id))

In [240]:
len(orig5Tweets)

737

Many of the conversation strands are in the df multiple times - from multiple persepctive!!! BEWARE

# Add info about the 5th level of replied to tweets to the DF

In [247]:
df['reply5OrigTime']=0
df['reply5OrigText']=0
df['reply5OrigReplyWhat']=0
df['reply5OrigReplyWho']=0

In [248]:
for id in orig5Tweets:
    df.loc[df.reply4OrigReplyWhat==id, 'reply5OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply4OrigReplyWhat==id, 'reply5OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply4OrigReplyWhat==id, 'reply5OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply4OrigReplyWhat==id, 'reply5OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [249]:
len(df[df.reply5OrigTime!=0])

797

In [254]:
len(df[(df.reply5OrigReplyWho!=0) & (df.reply5OrigReplyWho!='0')])

628

In [255]:
len(df[df.reply5OrigReplyWhat!=0])

596

In [256]:
orig6Tweets=list(set(df.reply5OrigReplyWhat[df.reply5OrigReplyWhat!=0]) & set(df.id))

In [257]:
len(orig6Tweets)

532

# Add info about the 6th level of replied to tweets to the DF

In [258]:
df['reply6OrigTime']=0
df['reply6OrigText']=0
df['reply6OrigReplyWhat']=0
df['reply6OrigReplyWho']=0

In [259]:
for id in orig6Tweets:
    df.loc[df.reply5OrigReplyWhat==id, 'reply6OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply5OrigReplyWhat==id, 'reply6OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply5OrigReplyWhat==id, 'reply6OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply5OrigReplyWhat==id, 'reply6OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [260]:
len(df[df.reply6OrigTime!=0])

571

In [261]:
len(df[(df.reply6OrigReplyWho!=0) & (df.reply6OrigReplyWho!='0')])

489

In [262]:
len(df[df.reply6OrigReplyWhat!=0])

466

In [265]:
orig7Tweets=list(set(df.reply6OrigReplyWhat[df.reply6OrigReplyWhat!=0]) & set(df.id))

In [266]:
len(orig7Tweets)

420

# Add info about the 7th level of replied to tweets to the DF

In [267]:
df['reply7OrigTime']=0
df['reply7OrigText']=0
df['reply7OrigReplyWhat']=0
df['reply7OrigReplyWho']=0

In [268]:
for id in orig7Tweets:
    df.loc[df.reply6OrigReplyWhat==id, 'reply7OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply6OrigReplyWhat==id, 'reply7OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply6OrigReplyWhat==id, 'reply7OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply6OrigReplyWhat==id, 'reply7OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [269]:
len(df[df.reply7OrigTime!=0])

450

In [270]:
len(df[(df.reply7OrigReplyWho!=0) & (df.reply7OrigReplyWho!='0')])

402

In [271]:
len(df[df.reply7OrigReplyWhat!=0])

390

In [272]:
orig8Tweets=list(set(df.reply7OrigReplyWhat[df.reply7OrigReplyWhat!=0]) & set(df.id))

In [273]:
len(orig8Tweets)

354

# Add info about the 8th level of replied to tweets to the DF

In [274]:
df['reply8OrigTime']=0
df['reply8OrigText']=0
df['reply8OrigReplyWhat']=0
df['reply8OrigReplyWho']=0

In [275]:
for id in orig8Tweets:
    df.loc[df.reply7OrigReplyWhat==id, 'reply8OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply7OrigReplyWhat==id, 'reply8OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply7OrigReplyWhat==id, 'reply8OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply7OrigReplyWhat==id, 'reply8OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [276]:
len(df[df.reply8OrigTime!=0])

377

In [277]:
len(df[(df.reply8OrigReplyWho!=0) & (df.reply8OrigReplyWho!='0')])

339

In [278]:
len(df[df.reply8OrigReplyWhat!=0])

333

In [279]:
orig9Tweets=list(set(df.reply8OrigReplyWhat[df.reply8OrigReplyWhat!=0]) & set(df.id))

In [280]:
len(orig9Tweets)

309

As the tweet has more and more originals, the originals are more likely to have originals - I guess we are selcting for certain users - people who go back and forth, do it a lot?

# Add info about the 9th level of replied to tweets to the DF

In [281]:
df['reply9OrigTime']=0
df['reply9OrigText']=0
df['reply9OrigReplyWhat']=0
df['reply9OrigReplyWho']=0

In [282]:
for id in orig9Tweets:
    df.loc[df.reply8OrigReplyWhat==id, 'reply9OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply8OrigReplyWhat==id, 'reply9OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply8OrigReplyWhat==id, 'reply9OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply8OrigReplyWhat==id, 'reply9OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [284]:
len(df[df.reply9OrigTime!=0])

326

In [285]:
len(df[(df.reply9OrigReplyWho!=0) & (df.reply9OrigReplyWho!='0')])

308

In [286]:
len(df[df.reply9OrigReplyWhat!=0])

303

In [287]:
orig10Tweets=list(set(df.reply9OrigReplyWhat[df.reply9OrigReplyWhat!=0]) & set(df.id))

In [288]:
len(orig10Tweets)

282

# Add info about the 10th level of replied to tweets to the DF

In [289]:
df['reply10OrigTime']=0
df['reply10OrigText']=0
df['reply10OrigReplyWhat']=0
df['reply10OrigReplyWho']=0

In [290]:
for id in orig10Tweets:
    df.loc[df.reply9OrigReplyWhat==id, 'reply10OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply9OrigReplyWhat==id, 'reply10OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply9OrigReplyWhat==id, 'reply10OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply9OrigReplyWhat==id, 'reply10OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [291]:
len(df[df.reply10OrigTime!=0])

298

In [292]:
len(df[(df.reply10OrigReplyWho!=0) & (df.reply10OrigReplyWho!='0')])

283

In [293]:
len(df[df.reply10OrigReplyWhat!=0])

279

In [294]:
orig11Tweets=list(set(df.reply10OrigReplyWhat[df.reply10OrigReplyWhat!=0]) & set(df.id))

In [295]:
len(orig11Tweets)

260

# Add info about the 11th level of replied to tweets to the DF

In [296]:
df['reply11OrigTime']=0
df['reply11OrigText']=0
df['reply11OrigReplyWhat']=0
df['reply11OrigReplyWho']=0

In [297]:
for id in orig11Tweets:
    df.loc[df.reply10OrigReplyWhat==id, 'reply11OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply10OrigReplyWhat==id, 'reply11OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply10OrigReplyWhat==id, 'reply11OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply10OrigReplyWhat==id, 'reply11OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [299]:
len(df[df.reply11OrigTime!=0])

275

In [300]:
len(df[(df.reply11OrigReplyWho!=0) & (df.reply11OrigReplyWho!='0')])

263

In [301]:
len(df[df.reply11OrigReplyWhat!=0])

259

In [302]:
orig12Tweets=list(set(df.reply11OrigReplyWhat[df.reply11OrigReplyWhat!=0]) & set(df.id))

In [303]:
len(orig12Tweets)

241

# Add info about the 12th level of replied to tweets to the DF

In [307]:
df['reply12OrigTime']=0
df['reply12OrigText']=0
df['reply12OrigReplyWhat']=0
df['reply12OrigReplyWho']=0

In [308]:
for id in orig12Tweets:
    df.loc[df.reply11OrigReplyWhat==id, 'reply12OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply11OrigReplyWhat==id, 'reply12OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply11OrigReplyWhat==id, 'reply12OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply11OrigReplyWhat==id, 'reply12OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [309]:
len(df[df.reply12OrigTime!=0])

255

In [310]:
len(df[(df.reply12OrigReplyWho!=0) & (df.reply12OrigReplyWho!='0')])

243

In [311]:
len(df[df.reply12OrigReplyWhat!=0])

241

In [312]:
orig13Tweets=list(set(df.reply12OrigReplyWhat[df.reply12OrigReplyWhat!=0]) & set(df.id))

In [313]:
len(orig13Tweets)

224

# Add info about the 13th level of replied to tweets to the DF

In [315]:
df['reply13OrigTime']=0
df['reply13OrigText']=0
df['reply13OrigReplyWhat']=0
df['reply13OrigReplyWho']=0

In [316]:
for id in orig13Tweets:
    df.loc[df.reply12OrigReplyWhat==id, 'reply13OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply12OrigReplyWhat==id, 'reply13OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply12OrigReplyWhat==id, 'reply13OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply12OrigReplyWhat==id, 'reply13OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [317]:
len(df[df.reply13OrigTime!=0])

237

In [318]:
len(df[(df.reply13OrigReplyWho!=0) & (df.reply13OrigReplyWho!='0')])

225

In [319]:
len(df[df.reply13OrigReplyWhat!=0])

219

In [321]:
df.user_id[df.reply13OrigReplyWhat!=0].unique()

array([ 74255540,  26957356, 335957725, 625674545, 278916697,  33195508,
       526539681,  21201205,  56464727, 329752526, 378383396, 426071565])

The same 12 users keep going back and forth

In [322]:
orig14Tweets=list(set(df.reply13OrigReplyWhat[df.reply13OrigReplyWhat!=0]) & set(df.id))

In [323]:
len(orig14Tweets)

207

# Add info about the 14th level of replied to tweets to the DF

In [330]:
df['reply14OrigTime']=0
df['reply14OrigText']=0
df['reply14OrigReplyWhat']=0
df['reply14OrigReplyWho']=0

In [331]:
for id in orig14Tweets:
    df.loc[df.reply13OrigReplyWhat==id, 'reply14OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply13OrigReplyWhat==id, 'reply14OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply13OrigReplyWhat==id, 'reply14OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply13OrigReplyWhat==id, 'reply14OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [333]:
len(df[df.reply14OrigTime!=0])

216

In [334]:
len(df[(df.reply14OrigReplyWho!=0) & (df.reply14OrigReplyWho!='0')])

205

In [335]:
len(df[df.reply14OrigReplyWhat!=0])

203

In [336]:
orig15Tweets=list(set(df.reply14OrigReplyWhat[df.reply14OrigReplyWhat!=0]) & set(df.id))

In [337]:
len(orig15Tweets)

192

In [339]:
df.user_id[df.reply14OrigReplyWhat!=0].unique()

array([ 26957356,  74255540, 625674545, 526539681,  33195508,  21201205,
       278916697, 329752526,  56464727, 426071565, 378383396])

# Add info about the 15th level of replied to tweets to the DF

In [356]:
df['reply15OrigTime']=0
df['reply15OrigText']=0
df['reply15OrigReplyWhat']=0
df['reply15OrigReplyWho']=0

In [357]:
for id in orig15Tweets:
    df.loc[df.reply14OrigReplyWhat==id, 'reply15OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply14OrigReplyWhat==id, 'reply15OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply14OrigReplyWhat==id, 'reply15OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply14OrigReplyWhat==id, 'reply15OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [358]:
len(df[df.reply15OrigTime!=0])

201

In [359]:
len(df[(df.reply15OrigReplyWho!=0) & (df.reply15OrigReplyWho!='0')])

190

In [361]:
len(df[df.reply15OrigReplyWhat!=0])

187

In [362]:
orig16Tweets=list(set(df.reply15OrigReplyWhat[df.reply15OrigReplyWhat!=0]) & set(df.id))

In [363]:
len(orig16Tweets)

177

In [370]:
del df['origTweet']
del df['origUser']
del df['origTime']
del df['origPlace']
del df['origUserCreated']

df[df.reply15OrigReplyWhat!=0].loc[30450]

Remove some columns not needed for this analsis to keep DF smaller

# Add info about the 16th level of replied to tweets to the DF

In [372]:
df['reply16OrigTime']=0
df['reply16OrigText']=0
df['reply16OrigReplyWhat']=0
df['reply16OrigReplyWho']=0

In [373]:
for id in orig16Tweets:
    df.loc[df.reply15OrigReplyWhat==id, 'reply16OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply15OrigReplyWhat==id, 'reply16OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply15OrigReplyWhat==id, 'reply16OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply15OrigReplyWhat==id, 'reply16OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [374]:
len(df[df.reply16OrigTime!=0])

185

In [375]:
len(df[(df.reply16OrigReplyWho!=0) & (df.reply16OrigReplyWho!='0')])

176

In [376]:
len(df[df.reply16OrigReplyWhat!=0])

173

In [377]:
orig17Tweets=list(set(df.reply16OrigReplyWhat[df.reply16OrigReplyWhat!=0]) & set(df.id))

In [378]:
len(orig17Tweets)

165

# Add info about the 17th level of replied to tweets to the DF

In [380]:
df['reply17OrigTime']=0
df['reply17OrigText']=0
df['reply17OrigReplyWhat']=0
df['reply17OrigReplyWho']=0

In [381]:
for id in orig17Tweets:
    df.loc[df.reply16OrigReplyWhat==id, 'reply17OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply16OrigReplyWhat==id, 'reply17OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply16OrigReplyWhat==id, 'reply17OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply16OrigReplyWhat==id, 'reply17OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [382]:
len(df[df.reply17OrigTime!=0])

170

In [383]:
len(df[(df.reply17OrigReplyWho!=0) & (df.reply17OrigReplyWho!='0')])

162

In [384]:
len(df[df.reply17OrigReplyWhat!=0])

160

In [385]:
orig18Tweets=list(set(df.reply17OrigReplyWhat[df.reply17OrigReplyWhat!=0]) & set(df.id))

In [386]:
len(orig18Tweets)

154

# Add info about the 18th level of replied to tweets to the DF

In [387]:
df['reply18OrigTime']=0
df['reply18OrigText']=0
df['reply18OrigReplyWhat']=0
df['reply18OrigReplyWho']=0

In [388]:
for id in orig18Tweets:
    df.loc[df.reply17OrigReplyWhat==id, 'reply18OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply17OrigReplyWhat==id, 'reply18OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply17OrigReplyWhat==id, 'reply18OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply17OrigReplyWhat==id, 'reply18OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [389]:
len(df[df.reply18OrigTime!=0])

158

In [390]:
len(df[(df.reply18OrigReplyWho!=0) & (df.reply18OrigReplyWho!='0')])

150

In [391]:
len(df[df.reply18OrigReplyWhat!=0])

149

In [392]:
orig19Tweets=list(set(df.reply18OrigReplyWhat[df.reply18OrigReplyWhat!=0]) & set(df.id))

In [394]:
len(orig19Tweets)

143

# Add info about the 19th level of replied to tweets to the DF

In [395]:
df['reply19OrigTime']=0
df['reply19OrigText']=0
df['reply19OrigReplyWhat']=0
df['reply19OrigReplyWho']=0

In [396]:
for id in orig19Tweets:
    df.loc[df.reply18OrigReplyWhat==id, 'reply19OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply18OrigReplyWhat==id, 'reply19OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply18OrigReplyWhat==id, 'reply19OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply18OrigReplyWhat==id, 'reply19OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [397]:
len(df[df.reply19OrigTime!=0])

147

In [398]:
len(df[(df.reply19OrigReplyWho!=0) & (df.reply19OrigReplyWho!='0')])

139

In [399]:
len(df[df.reply19OrigReplyWhat!=0])

138

In [400]:
orig20Tweets=list(set(df.reply19OrigReplyWhat[df.reply19OrigReplyWhat!=0]) & set(df.id))

In [401]:
len(orig20Tweets)

132

# Add info about the 20th level of replied to tweets to the DF

In [403]:
df['reply20OrigTime']=0
df['reply20OrigText']=0
df['reply20OrigReplyWhat']=0
df['reply20OrigReplyWho']=0

In [404]:
for id in orig20Tweets:
    df.loc[df.reply19OrigReplyWhat==id, 'reply20OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply19OrigReplyWhat==id, 'reply20OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply19OrigReplyWhat==id, 'reply20OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply19OrigReplyWhat==id, 'reply20OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [406]:
len(df[df.reply20OrigTime!=0])

136

In [408]:
len(df[(df.reply20OrigReplyWho!=0) & (df.reply20OrigReplyWho!='0')])

128

In [409]:
len(df[df.reply20OrigReplyWhat!=0])

127

In [410]:
orig21Tweets=list(set(df.reply20OrigReplyWhat[df.reply20OrigReplyWhat!=0]) & set(df.id))

In [411]:
len(orig21Tweets)

121

# Add info about the 21th level of replied to tweets to the DF

In [422]:
df['reply21OrigTime']=0
df['reply21OrigText']=0
df['reply21OrigReplyWhat']=0
df['reply21OrigReplyWho']=0

In [423]:
for id in orig21Tweets:
    df.loc[df.reply20OrigReplyWhat==id, 'reply21OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply20OrigReplyWhat==id, 'reply21OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply20OrigReplyWhat==id, 'reply21OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply20OrigReplyWhat==id, 'reply21OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [424]:
len(df[df.reply21OrigTime!=0])

124

In [425]:
len(df[(df.reply21OrigReplyWho!=0) & (df.reply21OrigReplyWho!='0')])

116

In [426]:
len(df[df.reply21OrigReplyWhat!=0])

115

In [427]:
orig22Tweets=list(set(df.reply21OrigReplyWhat[df.reply21OrigReplyWhat!=0]) & set(df.id))

In [428]:
len(orig22Tweets)

110

# Add info about the 22th level of replied to tweets to the DF

In [429]:
df['reply22OrigTime']=0
df['reply22OrigText']=0
df['reply22OrigReplyWhat']=0
df['reply22OrigReplyWho']=0

In [430]:
for id in orig22Tweets:
    df.loc[df.reply21OrigReplyWhat==id, 'reply22OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply21OrigReplyWhat==id, 'reply22OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply21OrigReplyWhat==id, 'reply22OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply21OrigReplyWhat==id, 'reply22OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [431]:
len(df[df.reply22OrigTime!=0])

112

In [432]:
len(df[(df.reply22OrigReplyWho!=0) & (df.reply22OrigReplyWho!='0')])

104

In [434]:
len(df[df.reply22OrigReplyWhat!=0])

104

In [435]:
orig23Tweets=list(set(df.reply22OrigReplyWhat[df.reply22OrigReplyWhat!=0]) & set(df.id))

In [436]:
len(orig23Tweets)

100

# Add info about the 23th level of replied to tweets to the DF

In [438]:
df['reply23OrigTime']=0
df['reply23OrigText']=0
df['reply23OrigReplyWhat']=0
df['reply23OrigReplyWho']=0

In [439]:
for id in orig23Tweets:
    df.loc[df.reply22OrigReplyWhat==id, 'reply23OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply22OrigReplyWhat==id, 'reply23OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply22OrigReplyWhat==id, 'reply23OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply22OrigReplyWhat==id, 'reply23OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [440]:
len(df[df.reply23OrigTime!=0])

100

In [441]:
len(df[(df.reply23OrigReplyWho!=0) & (df.reply23OrigReplyWho!='0')])

92

In [442]:
len(df[df.reply23OrigReplyWhat!=0])

92

In [443]:
orig24Tweets=list(set(df.reply23OrigReplyWhat[df.reply23OrigReplyWhat!=0]) & set(df.id))

In [444]:
len(orig24Tweets)

90

# Add info about the 24th level of replied to tweets to the DF

In [446]:
df['reply24OrigTime']=0
df['reply24OrigText']=0
df['reply24OrigReplyWhat']=0
df['reply24OrigReplyWho']=0

In [447]:
for id in orig24Tweets:
    df.loc[df.reply23OrigReplyWhat==id, 'reply24OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply23OrigReplyWhat==id, 'reply24OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply23OrigReplyWhat==id, 'reply24OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply23OrigReplyWhat==id, 'reply24OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [448]:
len(df[df.reply24OrigTime!=0])

90

In [449]:
len(df[(df.reply24OrigReplyWho!=0) & (df.reply24OrigReplyWho!='0')])

82

In [450]:
len(df[df.reply24OrigReplyWhat!=0])

82

In [451]:
orig25Tweets=list(set(df.reply24OrigReplyWhat[df.reply24OrigReplyWhat!=0]) & set(df.id))

In [452]:
len(orig25Tweets)

80

# Add info about the 25th level of replied to tweets to the DF

In [456]:
df['reply25OrigTime']=0
df['reply25OrigText']=0
df['reply25OrigReplyWhat']=0
df['reply25OrigReplyWho']=0

In [457]:
for id in orig25Tweets:
    df.loc[df.reply24OrigReplyWhat==id, 'reply25OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply24OrigReplyWhat==id, 'reply25OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply24OrigReplyWhat==id, 'reply25OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply24OrigReplyWhat==id, 'reply25OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [458]:
len(df[df.reply25OrigTime!=0])

80

In [459]:
len(df[(df.reply25OrigReplyWho!=0) & (df.reply25OrigReplyWho!='0')])

73

In [460]:
len(df[df.reply25OrigReplyWhat!=0])

73

In [461]:
orig26Tweets=list(set(df.reply25OrigReplyWhat[df.reply25OrigReplyWhat!=0]) & set(df.id))

In [462]:
len(orig26Tweets)

72

In [463]:
df.user_id[df.reply25OrigReplyWhat!=0].unique()

array([74255540, 26957356, 56464727])

# Add info about the 26th level of replied to tweets to the DF

In [464]:
df['reply26OrigTime']=0
df['reply26OrigText']=0
df['reply26OrigReplyWhat']=0
df['reply26OrigReplyWho']=0

In [465]:
for id in orig26Tweets:
    df.loc[df.reply25OrigReplyWhat==id, 'reply26OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply25OrigReplyWhat==id, 'reply26OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply25OrigReplyWhat==id, 'reply26OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply25OrigReplyWhat==id, 'reply26OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [466]:
orig27Tweets=list(set(df.reply26OrigReplyWhat[df.reply26OrigReplyWhat!=0]) & set(df.id))

In [467]:
len(orig27Tweets)

66

# Add info about the 27th level of replied to tweets to the DF

In [470]:
df['reply27OrigTime']=0
df['reply27OrigText']=0
df['reply27OrigReplyWhat']=0
df['reply27OrigReplyWho']=0

In [471]:
for id in orig27Tweets:
    df.loc[df.reply26OrigReplyWhat==id, 'reply27OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply26OrigReplyWhat==id, 'reply27OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply26OrigReplyWhat==id, 'reply27OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply26OrigReplyWhat==id, 'reply27OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [472]:
orig28Tweets=list(set(df.reply27OrigReplyWhat[df.reply27OrigReplyWhat!=0]) & set(df.id))

In [473]:
len(orig28Tweets)

60

In [474]:
df.username[df.reply27OrigReplyWhat!=0].unique()

array(['Megglezzz', 'DDougyy'], dtype=object)

# Add info about the 28th level of replied to tweets to the DF

In [475]:
df['reply28OrigTime']=0
df['reply28OrigText']=0
df['reply28OrigReplyWhat']=0
df['reply28OrigReplyWho']=0

In [476]:
for id in orig28Tweets:
    df.loc[df.reply27OrigReplyWhat==id, 'reply28OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply27OrigReplyWhat==id, 'reply28OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply27OrigReplyWhat==id, 'reply28OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply27OrigReplyWhat==id, 'reply28OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [477]:
orig29Tweets=list(set(df.reply28OrigReplyWhat[df.reply28OrigReplyWhat!=0]) & set(df.id))

In [478]:
len(orig29Tweets)

55

In [480]:
df.username[df.reply28OrigReplyWhat!=0].unique()

array(['DDougyy', 'Megglezzz'], dtype=object)

# Add info about the 29th level of replied to tweets to the DF

In [481]:
df['reply29OrigTime']=0
df['reply29OrigText']=0
df['reply29OrigReplyWhat']=0
df['reply29OrigReplyWho']=0

In [482]:
for id in orig29Tweets:
    df.loc[df.reply28OrigReplyWhat==id, 'reply29OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply28OrigReplyWhat==id, 'reply29OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply28OrigReplyWhat==id, 'reply29OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply28OrigReplyWhat==id, 'reply29OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [483]:
orig30Tweets=list(set(df.reply29OrigReplyWhat[df.reply29OrigReplyWhat!=0]) & set(df.id))

In [484]:
len(orig30Tweets)

50

# Add info about the 30th level of replied to tweets to the DF

In [486]:
df['reply30OrigTime']=0
df['reply30OrigText']=0
df['reply30OrigReplyWhat']=0
df['reply30OrigReplyWho']=0

In [487]:
for id in orig30Tweets:
    df.loc[df.reply29OrigReplyWhat==id, 'reply30OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply29OrigReplyWhat==id, 'reply30OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply29OrigReplyWhat==id, 'reply30OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply29OrigReplyWhat==id, 'reply30OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [488]:
orig31Tweets=list(set(df.reply30OrigReplyWhat[df.reply30OrigReplyWhat!=0]) & set(df.id))

In [489]:
len(orig31Tweets)

45

# Add info about the 31th level of replied to tweets to the DF

In [490]:
df['reply31OrigTime']=0
df['reply31OrigText']=0
df['reply31OrigReplyWhat']=0
df['reply31OrigReplyWho']=0

In [491]:
for id in orig31Tweets:
    df.loc[df.reply30OrigReplyWhat==id, 'reply31OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply30OrigReplyWhat==id, 'reply31OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply30OrigReplyWhat==id, 'reply31OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply30OrigReplyWhat==id, 'reply31OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [492]:
orig32Tweets=list(set(df.reply31OrigReplyWhat[df.reply31OrigReplyWhat!=0]) & set(df.id))

In [493]:
len(orig32Tweets)

40

# Add info about the 32th level of replied to tweets to the DF

In [494]:
df['reply32OrigTime']=0
df['reply32OrigText']=0
df['reply32OrigReplyWhat']=0
df['reply32OrigReplyWho']=0

In [495]:
for id in orig32Tweets:
    df.loc[df.reply31OrigReplyWhat==id, 'reply32OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply31OrigReplyWhat==id, 'reply32OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply31OrigReplyWhat==id, 'reply32OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply31OrigReplyWhat==id, 'reply32OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [496]:
orig33Tweets=list(set(df.reply32OrigReplyWhat[df.reply32OrigReplyWhat!=0]) & set(df.id))

In [497]:
len(orig33Tweets)

35

# Add info about the 33th level of replied to tweets to the DF

In [498]:
df['reply33OrigTime']=0
df['reply33OrigText']=0
df['reply33OrigReplyWhat']=0
df['reply33OrigReplyWho']=0

In [499]:
for id in orig33Tweets:
    df.loc[df.reply32OrigReplyWhat==id, 'reply33OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply32OrigReplyWhat==id, 'reply33OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply32OrigReplyWhat==id, 'reply33OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply32OrigReplyWhat==id, 'reply33OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [500]:
orig34Tweets=list(set(df.reply33OrigReplyWhat[df.reply33OrigReplyWhat!=0]) & set(df.id))

In [501]:
len(orig34Tweets)

30

# Add info about the 34th level of replied to tweets to the DF

In [502]:
df['reply34OrigTime']=0
df['reply34OrigText']=0
df['reply34OrigReplyWhat']=0
df['reply34OrigReplyWho']=0

In [503]:
for id in orig34Tweets:
    df.loc[df.reply33OrigReplyWhat==id, 'reply34OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply33OrigReplyWhat==id, 'reply34OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply33OrigReplyWhat==id, 'reply34OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply33OrigReplyWhat==id, 'reply34OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [504]:
orig35Tweets=list(set(df.reply34OrigReplyWhat[df.reply34OrigReplyWhat!=0]) & set(df.id))

In [505]:
len(orig35Tweets)

26

In [506]:
df.username[df.reply33OrigReplyWhat!=0].unique()

array(['Megglezzz', 'DDougyy'], dtype=object)

# Add info about the 35th level of replied to tweets to the DF

In [507]:
df['reply35OrigTime']=0
df['reply35OrigText']=0
df['reply35OrigReplyWhat']=0
df['reply35OrigReplyWho']=0

In [508]:
for id in orig35Tweets:
    df.loc[df.reply34OrigReplyWhat==id, 'reply35OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply34OrigReplyWhat==id, 'reply35OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply34OrigReplyWhat==id, 'reply35OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply34OrigReplyWhat==id, 'reply35OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [509]:
orig36Tweets=list(set(df.reply35OrigReplyWhat[df.reply35OrigReplyWhat!=0]) & set(df.id))

In [510]:
len(orig36Tweets)

22

# Add info about the 36th level of replied to tweets to the DF

In [511]:
df['reply36OrigTime']=0
df['reply36OrigText']=0
df['reply36OrigReplyWhat']=0
df['reply36OrigReplyWho']=0

In [512]:
for id in orig36Tweets:
    df.loc[df.reply35OrigReplyWhat==id, 'reply36OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply35OrigReplyWhat==id, 'reply36OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply35OrigReplyWhat==id, 'reply36OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply35OrigReplyWhat==id, 'reply36OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [513]:
orig37Tweets=list(set(df.reply36OrigReplyWhat[df.reply36OrigReplyWhat!=0]) & set(df.id))

In [514]:
len(orig37Tweets)

18

# Add info about the 37th level of replied to tweets to the DF

In [515]:
df['reply37OrigTime']=0
df['reply37OrigText']=0
df['reply37OrigReplyWhat']=0
df['reply37OrigReplyWho']=0

In [516]:
for id in orig37Tweets:
    df.loc[df.reply36OrigReplyWhat==id, 'reply37OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply36OrigReplyWhat==id, 'reply37OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply36OrigReplyWhat==id, 'reply37OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply36OrigReplyWhat==id, 'reply37OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [517]:
orig38Tweets=list(set(df.reply37OrigReplyWhat[df.reply37OrigReplyWhat!=0]) & set(df.id))

In [518]:
len(orig38Tweets)

14

# Add info about the 38th level of replied to tweets to the DF

In [519]:
df['reply38OrigTime']=0
df['reply38OrigText']=0
df['reply38OrigReplyWhat']=0
df['reply38OrigReplyWho']=0

In [520]:
for id in orig38Tweets:
    df.loc[df.reply37OrigReplyWhat==id, 'reply38OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply37OrigReplyWhat==id, 'reply38OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply37OrigReplyWhat==id, 'reply38OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply37OrigReplyWhat==id, 'reply38OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [521]:
orig39Tweets=list(set(df.reply38OrigReplyWhat[df.reply38OrigReplyWhat!=0]) & set(df.id))

In [522]:
len(orig39Tweets)

11

# Add info about the 39th level of replied to tweets to the DF

In [523]:
df['reply39OrigTime']=0
df['reply39OrigText']=0
df['reply39OrigReplyWhat']=0
df['reply39OrigReplyWho']=0

In [524]:
for id in orig39Tweets:
    df.loc[df.reply38OrigReplyWhat==id, 'reply39OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply38OrigReplyWhat==id, 'reply39OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply38OrigReplyWhat==id, 'reply39OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply38OrigReplyWhat==id, 'reply39OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [525]:
orig40Tweets=list(set(df.reply39OrigReplyWhat[df.reply39OrigReplyWhat!=0]) & set(df.id))

In [526]:
len(orig40Tweets)

8

# Add info about the 40th level of replied to tweets to the DF

In [527]:
df['reply40OrigTime']=0
df['reply40OrigText']=0
df['reply40OrigReplyWhat']=0
df['reply40OrigReplyWho']=0

In [528]:
for id in orig40Tweets:
    df.loc[df.reply39OrigReplyWhat==id, 'reply40OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply39OrigReplyWhat==id, 'reply40OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply39OrigReplyWhat==id, 'reply40OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply39OrigReplyWhat==id, 'reply40OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [529]:
orig41Tweets=list(set(df.reply40OrigReplyWhat[df.reply40OrigReplyWhat!=0]) & set(df.id))

In [530]:
len(orig41Tweets)

5

# Add info about the 41th level of replied to tweets to the DF

In [531]:
df['reply41OrigTime']=0
df['reply41OrigText']=0
df['reply41OrigReplyWhat']=0
df['reply41OrigReplyWho']=0

In [532]:
for id in orig41Tweets:
    df.loc[df.reply40OrigReplyWhat==id, 'reply41OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply40OrigReplyWhat==id, 'reply41OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply40OrigReplyWhat==id, 'reply41OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply40OrigReplyWhat==id, 'reply41OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [533]:
orig42Tweets=list(set(df.reply41OrigReplyWhat[df.reply41OrigReplyWhat!=0]) & set(df.id))

In [534]:
len(orig42Tweets)

3

# Add info about the 42th level of replied to tweets to the DF

In [535]:
df['reply42OrigTime']=0
df['reply42OrigText']=0
df['reply42OrigReplyWhat']=0
df['reply42OrigReplyWho']=0

In [536]:
for id in orig42Tweets:
    df.loc[df.reply41OrigReplyWhat==id, 'reply42OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply41OrigReplyWhat==id, 'reply42OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply41OrigReplyWhat==id, 'reply42OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply41OrigReplyWhat==id, 'reply42OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [537]:
orig43Tweets=list(set(df.reply42OrigReplyWhat[df.reply42OrigReplyWhat!=0]) & set(df.id))

In [538]:
len(orig43Tweets)

1

In [539]:
df.username[df.reply42OrigReplyWhat!=0]

2454450    DDougyy
Name: username, dtype: object

# Add info about the 43th level of replied to tweets to the DF

In [540]:
df['reply43OrigTime']=0
df['reply43OrigText']=0
df['reply43OrigReplyWhat']=0
df['reply43OrigReplyWho']=0

In [541]:
for id in orig43Tweets:
    df.loc[df.reply42OrigReplyWhat==id, 'reply43OrigTime']=(df.loc[df.id==id,'date']).values[0]
    df.loc[df.reply42OrigReplyWhat==id, 'reply43OrigText']=(df.loc[df.id==id,'text']).values[0]
    df.loc[df.reply42OrigReplyWhat==id, 'reply43OrigReplyWhat']=(df.loc[df.id==id,'replyWhat']).values[0]
    df.loc[df.reply42OrigReplyWhat==id, 'reply43OrigReplyWho']=(df.loc[df.id==id,'replyWho']).values[0]

In [542]:
orig44Tweets=list(set(df.reply43OrigReplyWhat[df.reply43OrigReplyWhat!=0]) & set(df.id))

In [543]:
len(orig44Tweets)

0

#  Get and save all teh convos

In [545]:
reps=df[df.replyWhat!=0]

In [547]:
reps.to_csv('../data/bbAllConvoLinesDF.csv',sep=';', index=False)